In [1]:
import shutil

# importing the sys module
import sys        
 
# appending the directory of mod.py
# in the sys.path list
sys.path.append('../')   

import json
import os
import random
from collections import defaultdict

import torch
from PIL import Image
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.optim import lr_scheduler
from torch.optim import Adam
import numpy as np
from sklearn.model_selection import train_test_split

from datetime import datetime

from Common.Optimizers.RangerLars import RangerLars
from Common.Schedulers.GradualWarmupScheduler import GradualWarmupScheduler

import range_classification__config as cfg

from range_classification__dataset import RangeClassificationSameUniformDiffDataset, RangeClassificationMixedUniformDiffDataset, RangeClassificationMimicDiffDataset, get_error_constrained_dataset
from Common.Datasets.Morph2.data_parser import DataParser
from range_classification__train import train_diff_cls_model_iter


from range_classification__model import RangeClassificationModel, RangeClassificationDeepModel

from tqdm import tqdm

from sklearn import utils

import matplotlib.pyplot as plt

import pandas as pd


from sklearn import metrics
import seaborn
import matplotlib.pyplot as plt
from condor_pytorch.metrics import mean_absolute_error
from condor_pytorch.dataset import levels_from_labelbatch

from scipy.stats import pearsonr

In [2]:
cfg.MULTI_GPU

True

In [3]:
age_diff_model_path = 'D:/projects/age_estimation/RangeClassPipeline/weights/Morph2Diff/unified/iter/time_01_09_2022_13_54_07'
age_diff_model_file = os.path.join(age_diff_model_path, "weights.pt")

from collections import OrderedDict
file_dict = torch.load(age_diff_model_file)
new_file_dict = OrderedDict()
for x in file_dict:
    new_file_dict[x.split('module.')[1]] = file_dict[x] #new_file_dict[]
    
new_file_dict

FileNotFoundError: [Errno 2] No such file or directory: 'D:/projects/age_estimation/RangeClassPipeline/weights/Morph2Diff/unified/iter/time_01_09_2022_13_54_07\\weights.pt'

In [ ]:
def get_age_diff_range_predictor(age_interval, min_age, max_age, age_radius, device, age_diff_range_model):
    #age_diff_model_inst = age_diff_model(age_interval, min_age, max_age, age_radius, device, deep, num_references, config_type, added_embed_layer_size, diff_embed_layer_size, is_ordinal_reg)
    age_diff_range_model_inst = age_diff_range_model(age_interval, min_age, max_age, age_radius, device) #, config_type, added_embed_layer_size, diff_embed_layer_size, is_ordinal_reg)


    if cfg.USE_GPU and cfg.MULTI_GPU:
        if torch.cuda.device_count() > 1:
            print("Using multiple GPUs (" + str(torch.cuda.device_count()) + ")")
            age_diff_model_inst = torch.nn.DataParallel(age_diff_range_model_inst)



    print(age_diff_range_model_inst)
    age_diff_model_path = 'D:/projects/age_estimation/RangeClassPipeline/weights/Morph2Diff/unified/iter/time_01_09_2022_13_54_07'
    age_diff_model_file = os.path.join(age_diff_model_path, "weights.pt")

    file_dict = torch.load(age_diff_model_file)
    new_file_dict = OrderedDict()
    for x in file_dict:
        new_file_dict[x.split('module.')[1]] = file_dict[x] #new_file_dict[]



    age_diff_range_model_inst.load_state_dict(new_file_dict) #torch.load(age_diff_model_file))#, strict=False)
    age_diff_range_model_inst.to(device)

    return age_diff_range_model_inst

In [ ]:
#####################################################
#           Preparations
#####################################################

torch.manual_seed(cfg.RANDOM_SEED)
np.random.seed(cfg.RANDOM_SEED)
random.seed(cfg.RANDOM_SEED)

if cfg.USE_GPU:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")

print(device)

torch.cuda.empty_cache()

#####################################################
#           Data Loading
#####################################################

with open("im2age_map_test.json", 'r') as im2age_map_test_f:
    im2age_map_test = json.load(im2age_map_test_f)

# Load data
data_parser = DataParser('../Common/Datasets/Morph2/aligned_data/aligned_dataset_with_metadata_uint8.hdf5', small_data=cfg.SMALL_DATA)
data_parser.initialize_data()


x_train, y_train, x_test, y_test = data_parser.x_train,	data_parser.y_train, data_parser.x_test, data_parser.y_test,
if cfg.RANDOM_SPLIT:
    all_images = np.concatenate((x_train, x_test), axis=0)
    all_labels = np.concatenate((y_train, y_test), axis=0)

    x_train, x_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=cfg.TEST_SIZE_FOR_RS, random_state=cfg.RANDOM_SEED)


# shuffle basic aligned test
# x_test_shuffled
# y_test_shuffled


transf = transforms.Compose([
        transforms.RandomResizedCrop(224, (0.9, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1
        )], p=0.5),
        transforms.RandomApply([transforms.RandomAffine(
            degrees=10,
            translate=(0.1, 0.1),
            scale=(0.9, 1.1),
            shear=5,
            resample=Image.BICUBIC
        )], p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.5)
    ])

train_ds = RangeClassificationSameUniformDiffDataset(
    data_set_images=x_train,
    data_set_metadata=y_train,
    min_age=cfg.MIN_AGE,
    age_interval=cfg.AGE_INTERVAL,
    max_age=cfg.MAX_AGE,
    transform=transf,
    copies=1, 
    age_radius=cfg.AGE_RADIUS,
    age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
    age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI
)

print("Training (q trn, r trn) set size: " + str(len(train_ds)))



x_test_filtered, y_test_filtered, batst_set_filtered_indexes = get_error_constrained_dataset(orig_dataset_images=x_test, 
                                                                                                orig_dataset_metadata=y_test,
                                                                                                age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
                                                                                                age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
                                                                                                im2age_map_batst=im2age_map_test)

#apref = Age Predict Reference (dataset is based on (q,r) pairs where r's age is what AgePredict model returns on q)
test_apref_ds = RangeClassificationMimicDiffDataset(
    batrn_set_images=x_train,
    batrn_set_metadata=y_train,
    batst_set_images=x_test,#x_test_filtered,
    batst_set_metadata=y_test,#y_test_filtered,
    batst_set_indexes=batst_set_filtered_indexes,
    im2age_map_batst=im2age_map_test,
    min_age=cfg.MIN_AGE,
    age_interval=cfg.AGE_INTERVAL,
    max_age=cfg.MAX_AGE,
    transform=transf,
    copies=1, 
    age_radius=cfg.AGE_RADIUS
)

print("Testing (q tst where AgePredict(q)  {age_diff_learn_radius_lo} <= error <= {age_diff_learn_radius_hi}, r trn where age(r)=AgePredict(q)) set size: ".format(age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI) + str(len(test_apref_ds)))


image_datasets = {
    'val_apref_ds': test_apref_ds
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['val_apref_ds']}

data_loaders = {
    'val_apref_ds': DataLoader(test_apref_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True)
}

In [ ]:
model = RangeClassificationModel

In [ ]:
# if cfg.USE_GPU and cfg.MULTI_GPU:
#         if torch.cuda.device_count() > 1:
#             print("Using multiple GPUs (" + str(torch.cuda.device_count()) + ")")
#             model = torch.nn.DataParallel(model)

# model.to(device)

In [ ]:
def diff_range_pipeline_confusion_matrix_analysis(model, val_data_loader, device, dataset_size):
    print('running on validation set...')

    pred_full = np.array([])
    labels_cls = np.array([])

    limit = -1
    for i, batch in enumerate(tqdm(val_data_loader)):
        inputs = batch['image_vec'].to(device) #batch['image_vec'][:,:2,:,:,:].to(device)
        labels = batch['label'].to(device).float()
        age_diff = batch['age_diff'].to(device).float()  

        #with torch.no_grad():
        classification_logits = model(inputs)
        
        _, age_diff_pred_hard = torch.max(classification_logits, 1)


        pred = age_diff_pred_hard.detach().to('cpu').numpy()

        labels_cls = np.concatenate((labels_cls, labels.to('cpu').numpy()), axis=0)

        pred_full = np.concatenate((pred_full, pred), axis=0)




        if limit != -1:
            if i == limit-1:
                break



    # import pdb
    # pdb.set_trace()

    fig, ax = plt.subplots()
    
    print(len(pred_full))
    print()

    class_names = [0,1,2] #range(-cfg.AGE_RADIUS, cfg.AGE_RADIUS + 1)]
    cnf_matrix = metrics.confusion_matrix(labels_cls, pred_full, labels=[0,1,2])
    seaborn.heatmap(pd.DataFrame(data=cnf_matrix, index=class_names, columns=class_names), annot=True, cmap="YlGnBu", fmt='g')

    ax.xaxis.set_label_position('top')
    plt.tight_layout()
    plt.xlabel('Predicted Label')
    plt.ylabel('Actual Label')

    plt.plot()
    plt.show()

In [ ]:
age_diff_predictor = get_age_diff_range_predictor(cfg.AGE_INTERVAL, cfg.MIN_AGE, cfg.MAX_AGE, cfg.AGE_RADIUS, device, model)
age_diff_predictor.eval()

In [ ]:
age_diff_predictor

In [ ]:
val_data_loader = data_loaders['val_apref_ds']
dataset_size = dataset_sizes['val_apref_ds']
diff_range_pipeline_confusion_matrix_analysis(age_diff_predictor, val_data_loader, device, dataset_size)

In [ ]:
8801/10606

In [ ]:
# x_test_filtered, y_test_filtered, batst_set_filtered_indexes = get_error_constrained_dataset(orig_dataset_images=x_test, 
#                                                                                                 orig_dataset_metadata=y_test,
#                                                                                                 age_diff_learn_radius_lo=0,
#                                                                                                 age_diff_learn_radius_hi=2,
#                                                                                                 im2age_map_batst=im2age_map_test)
# print(len(x_test_filtered))

# x_test_filtered, y_test_filtered, batst_set_filtered_indexes = get_error_constrained_dataset(orig_dataset_images=x_test, 
#                                                                                                 orig_dataset_metadata=y_test,
#                                                                                                 age_diff_learn_radius_lo=2,
#                                                                                                 age_diff_learn_radius_hi=4,
#                                                                                                 im2age_map_batst=im2age_map_test)
# print(len(x_test_filtered))